In [1]:
import pandas as pd
auto_df = pd.read_csv("Auto.csv")

In [2]:
mpg_median = auto_df['mpg'].median()

# Create the binary variable mpg01
auto_df['mpg01'] = (auto_df['mpg'] > mpg_median).astype(bool)

In [3]:
def find_primary_keys(df):
    """
    Finds all minimal candidate primary keys in the DataFrame.
    Returns a list of tuples, each tuple is a set of column names forming a primary key.
    """
    from itertools import combinations
    cols = df.columns
    candidate_keys = []

    for r in range(1, len(cols)+1):  # from 1 column up to all columns
        for combo in combinations(cols, r):
            # Skip if any column in combo has nulls
            if df[list(combo)].isnull().any().any():
                continue
            
            # Check uniqueness
            if not df.duplicated(subset=combo).any():
                # Check minimality: no existing candidate key should be subset
                if not any(set(key).issubset(combo) for key in candidate_keys):
                    candidate_keys.append(combo)

    return candidate_keys

primary_keys = find_primary_keys(auto_df)
print("Candidate primary keys:", primary_keys)

Candidate primary keys: [('mpg', 'year', 'name'), ('weight', 'acceleration', 'year'), ('weight', 'year', 'name'), ('acceleration', 'year', 'name'), ('mpg', 'displacement', 'weight', 'year'), ('mpg', 'horsepower', 'weight', 'year'), ('mpg', 'weight', 'year', 'origin')]


In [4]:
def is_primary_key(df, cols):
    from itertools import combinations
    # 1. Check for nulls
    if df[cols].isnull().any().any():
        print("Contains nulls")
        return False
    
    # 2. Check uniqueness
    if df.duplicated(subset=cols).any():
        print("Duplicates found")
        return False
    
    # 3. Check minimality
    for r in range(1, len(cols)):
        for subset in combinations(cols, r):
            if not df.duplicated(subset=subset).any():
                print("Not minimal, subset {subset} is also unique")
                return False, 
    
    # All checks passed
    return True, "Valid composite key"

primary_key_columns = ['year', 'name','weight']
is_primary_key(auto_df, primary_key_columns)

(True, 'Valid composite key')

In [5]:
auto_df['Usage'] = "Public"
auto_df['ID'] = auto_df[primary_key_columns].astype(str).agg('_'.join, axis=1)
auto_df = auto_df.set_index(['ID'])

In [6]:
auto_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name', 'mpg01', 'Usage'],
      dtype='object')

In [7]:
auto_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg01,Usage
ID,,,,,,,,,,,
70_chevrolet chevelle malibu_3504,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,False,Public
70_buick skylark 320_3693,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,False,Public
70_plymouth satellite_3436,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,False,Public
70_amc rebel sst_3433,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,False,Public
70_ford torino_3449,17.0,8,302.0,140,3449,10.5,70,1,ford torino,False,Public
...,...,...,...,...,...,...,...,...,...,...,...
82_ford mustang gl_2790,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl,True,Public
82_vw pickup_2130,44.0,4,97.0,52,2130,24.6,82,2,vw pickup,True,Public
82_dodge rampage_2295,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage,True,Public


In [8]:
# Move ID to the first column
auto_df = auto_df[['mpg01', 'Usage']]
# Write to CSV
auto_df.to_csv('auto_with_mpg_class.csv', index=True)

In [9]:
auto_df[['mpg01']].to_csv('sandbox.csv',index=True)